# Test of GAT
- use DGL
- predict trilets
- valid, test data are not in the training dataset

In [4]:
import dgl
import json
import torch
import torch as th
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [5]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

print(device)


cuda:0


## Fix the seed

In [6]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [7]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [8]:
datasets = ['train', 'valid', 'test']
dataset_data = {}

for dataset_name in tqdm(datasets):
#     file_path = f"../../data_processing/dgl/data/test_graph/repeated_{dataset_name}.jsonl"
    file_path = f"../../data_processing/dgl/data/test_triplet/repeated_test_{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")


  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data/test_triplet/repeated_test_train.jsonl


230000it [00:44, 5131.37it/s]
 33%|███▎      | 1/3 [00:44<01:29, 44.83s/it]

../../data_processing/dgl/data/test_triplet/repeated_test_valid.jsonl


9it [00:00, 9513.29it/s]


../../data_processing/dgl/data/test_triplet/repeated_test_test.jsonl


6it [00:00, 9258.95it/s]
100%|██████████| 3/3 [00:44<00:00, 14.95s/it]

Datasets loaded!


- choose batch size

In [9]:
def create_dataloaders(batch_size, shuffle=False):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(4)
# dataloaders = create_dataloaders(16)

- Turn the print message to a log file

In [10]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GAT.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0816_10:29_GAT.log


### Model

In [11]:
class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, dropout_prob=0.25):
        super(GAT, self).__init__()
        
        # do not check the zero in_degree since we have all the complete graph
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads, activation=F.relu, allow_zero_in_degree=True)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=num_heads, allow_zero_in_degree=True)
        
        # Adding Batch Normalization after each GAT layer
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim * num_heads)
        self.batchnorm2 = nn.BatchNorm1d(out_dim)
        
        # Adding Dropout for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, g, h):
        # Apply GAT layers
        h = self.layer1(g, h)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h).squeeze(1)
        
        # Store the output as a new node feature
        g.ndata['h_out'] = h

        # Use mean pooling to aggregate this new node feature
        h_agg = dgl.mean_nodes(g, feat='h_out')
        return h_agg

    

- Model Forward  

In [12]:
def model_fn(data, model, criterion, device, count=1, type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
    logits = logits.mean(dim=1)
    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability.
    preds = logits.argmax(1)
    
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())
    
    if type == 'validation':
        add_log_msg(f"labels of validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of validation: {preds} {preds.shape}")
        
    elif type == 'test':
        add_log_msg(f"labels of test: {labels} {labels.shape}")
        add_log_msg(f"predicted of test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [10]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
torch.save(model.state_dict(), 'model1_initial/initial_weight.pth')

In [11]:
model.layer1.fc.weight

Parameter containing:
tensor([[-0.1806, -0.0598,  0.0091,  ...,  0.0719,  0.2496,  0.0873],
        [ 0.1694, -0.0015, -0.0139,  ...,  0.0147,  0.0892,  0.0146],
        [ 0.0969, -0.0595, -0.0115,  ..., -0.0474,  0.0529, -0.0565],
        ...,
        [-0.0433, -0.2248,  0.3002,  ...,  0.0850,  0.1621,  0.0422],
        [ 0.2097, -0.2492,  0.0612,  ..., -0.0041,  0.0365, -0.1483],
        [ 0.0971, -0.2221,  0.1652,  ..., -0.1312, -0.2610,  0.0077]],
       requires_grad=True)

- Check if model really load the model_dict

In [12]:
model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
model.layer1.fc.weight

Parameter containing:
tensor([[-0.1806, -0.0598,  0.0091,  ...,  0.0719,  0.2496,  0.0873],
        [ 0.1694, -0.0015, -0.0139,  ...,  0.0147,  0.0892,  0.0146],
        [ 0.0969, -0.0595, -0.0115,  ..., -0.0474,  0.0529, -0.0565],
        ...,
        [-0.0433, -0.2248,  0.3002,  ...,  0.0850,  0.1621,  0.0422],
        [ 0.2097, -0.2492,  0.0612,  ..., -0.0041,  0.0365, -0.1483],
        [ 0.0971, -0.2221,  0.1652,  ..., -0.1312, -0.2610,  0.0077]],
       requires_grad=True)

### test of valid and test part is ``graph``

In [37]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))

model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)

criterion = nn.CrossEntropyLoss()
total_steps = 180

# save the best model
best_val_loss = float('inf')
patience = 500  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    count = 0 
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        
        count += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, count, type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()
        num_batches += 1
        
#     scheduler.step()
    add_log_msg(f"total count: {count}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for batched_g in dataloaders['valid']:
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, type=='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
#     print(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}')
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}')
    
    
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
#         torch.save(model.state_dict(), 'best_model.pth')
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f"../checkpoint_GAT/best_model_{epoch}.pt")
    
    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("Early stopping")
            break

            
# Testing Part
model.eval()
total = 0
correct = 0

with torch.no_grad():
    for data in dataloaders['test']:
        loss, accuracy, predicted = model_fn(data, model, criterion, device, type=='test')
        labels = data[1].to(device)  # Assuming labels are the second element in the tuple
        
#         print(f"labels: {labels}", labels.shape)
#         print(f"predicted: {predicted}", predicted.shape)
        
        add_log_msg(f"labels: {labels} {labels.shape}")
        add_log_msg(f"predicted: {predicted} {predicted.shape}")
        
        total += labels.size(0) # label.size(0) is the batch size
        correct += (predicted == labels).sum().item() 
        # (predicted == labels).sum() would return how many of them are equal; 
        # .item() would make the tensor to the regular value
        
#     print('Test Accuracy: %d %%' % (100 * correct / total))
add_log_msg(f'Test Accuracy: {100 * correct / total} %%')

  0%|          | 0/180 [00:00<?, ?it/s]

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

08/15/2023, 18:53:23# labels of 5000: tensor([84, 82, 86, 87], device='cuda:0') torch.Size([4])
08/15/2023, 18:53:23# predicted of 5000: tensor([76, 76, 76, 76], device='cuda:0') torch.Size([4])
08/15/2023, 18:54:46# labels of 10000: tensor([ 76,  70,  71, 119], device='cuda:0') torch.Size([4])
08/15/2023, 18:54:46# predicted of 10000: tensor([ 76,  76,  76, 119], device='cuda:0') torch.Size([4])
08/15/2023, 18:56:13# labels of 15000: tensor([87, 88, 90, 89], device='cuda:0') torch.Size([4])
08/15/2023, 18:56:13# predicted of 15000: tensor([119,  71,  76,  76], device='cuda:0') torch.Size([4])
08/15/2023, 18:57:38# labels of 20000: tensor([119,  78,  79,  77], device='cuda:0') torch.Size([4])
08/15/2023, 18:57:38# predicted of 20000: tensor([119,  70,  76,  76], device='cuda:0') torch.Size([4])
08/15/2023, 18:59:01# labels of 25000: tensor([ 89, 100, 101, 105], device='cuda:0') torch.Size([4])
08/15/2023, 18:59:01# predicted of 25000: tensor([76, 76, 76, 76], device='cuda:0') torch.Siz

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

08/15/2023, 19:01:34# labels of 5000: tensor([84, 82, 86, 87], device='cuda:0') torch.Size([4])
08/15/2023, 19:01:34# predicted of 5000: tensor([ 71,  76,  79, 103], device='cuda:0') torch.Size([4])
08/15/2023, 19:02:59# labels of 10000: tensor([ 76,  70,  71, 119], device='cuda:0') torch.Size([4])
08/15/2023, 19:02:59# predicted of 10000: tensor([ 76,  76,  76, 119], device='cuda:0') torch.Size([4])
08/15/2023, 19:04:24# labels of 15000: tensor([87, 88, 90, 89], device='cuda:0') torch.Size([4])
08/15/2023, 19:04:24# predicted of 15000: tensor([ 76, 101,  76,  86], device='cuda:0') torch.Size([4])
08/15/2023, 19:05:49# labels of 20000: tensor([119,  78,  79,  77], device='cuda:0') torch.Size([4])
08/15/2023, 19:05:49# predicted of 20000: tensor([119, 104,  76,  76], device='cuda:0') torch.Size([4])
08/15/2023, 19:07:15# labels of 25000: tensor([ 89, 100, 101, 105], device='cuda:0') torch.Size([4])
08/15/2023, 19:07:15# predicted of 25000: tensor([ 78, 100,  76, 101], device='cuda:0') t

...

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

08/15/2023, 23:18:32# labels of 5000: tensor([84, 82, 86, 87], device='cuda:0') torch.Size([4])
08/15/2023, 23:18:32# predicted of 5000: tensor([ 85,  76,  78, 105], device='cuda:0') torch.Size([4])
08/15/2023, 23:19:56# labels of 10000: tensor([ 76,  70,  71, 119], device='cuda:0') torch.Size([4])
08/15/2023, 23:19:56# predicted of 10000: tensor([ 76,  79,  76, 119], device='cuda:0') torch.Size([4])
08/15/2023, 23:21:21# labels of 15000: tensor([87, 88, 90, 89], device='cuda:0') torch.Size([4])
08/15/2023, 23:21:21# predicted of 15000: tensor([ 85,  88, 100, 105], device='cuda:0') torch.Size([4])
08/15/2023, 23:22:47# labels of 20000: tensor([119,  78,  79,  77], device='cuda:0') torch.Size([4])
08/15/2023, 23:22:47# predicted of 20000: tensor([119,  87,  76,  70], device='cuda:0') torch.Size([4])
08/15/2023, 23:24:11# labels of 25000: tensor([ 89, 100, 101, 105], device='cuda:0') torch.Size([4])
08/15/2023, 23:24:11# predicted of 25000: tensor([ 81,  85,  76, 104], device='cuda:0') t

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

08/15/2023, 23:26:35# labels of 5000: tensor([84, 82, 86, 87], device='cuda:0') torch.Size([4])
08/15/2023, 23:26:35# predicted of 5000: tensor([100,  76,  88,  89], device='cuda:0') torch.Size([4])
08/15/2023, 23:28:01# labels of 10000: tensor([ 76,  70,  71, 119], device='cuda:0') torch.Size([4])
08/15/2023, 23:28:01# predicted of 10000: tensor([ 76,  82,  76, 119], device='cuda:0') torch.Size([4])
08/15/2023, 23:29:27# labels of 15000: tensor([87, 88, 90, 89], device='cuda:0') torch.Size([4])
08/15/2023, 23:29:27# predicted of 15000: tensor([100, 104,  85,  85], device='cuda:0') torch.Size([4])
08/15/2023, 23:30:50# labels of 20000: tensor([119,  78,  79,  77], device='cuda:0') torch.Size([4])
08/15/2023, 23:30:50# predicted of 20000: tensor([119, 104,  79, 103], device='cuda:0') torch.Size([4])


KeyboardInterrupt: 

### test of valid and test part is ``triplet`` not graph

- 23 APs x 5000 times

In [70]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))

model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)

criterion = nn.CrossEntropyLoss()
total_steps = 18

# save the best model
best_val_loss = float('inf')
patience = 3  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    count = 0 
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        
        count += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, count)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()
        num_batches += 1
        
#     scheduler.step()
    print(f"total count: {count}")
    
    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    print(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for batched_g in dataloaders['valid']:
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, count=0)
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    print(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}')
    
    
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
#         torch.save(model.state_dict(), 'best_model.pth')
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f"../checkpoint_GAT/best_model_{epoch}.pt")
    
    else:
        waiting += 1
        if waiting >= patience:
            print("Early stopping")
            break

# Testing Part
model.eval()
total = 0
correct = 0

with torch.no_grad():
    for data in dataloaders['test']:
        loss, accuracy, predicted = model_fn(data, model, criterion, device)
        labels = data[1].to(device)  # Assuming labels are the second element in the tuple
        
        print(f"labels: {labels}", labels.shape)
        print(f"predicted: {predicted}", predicted.shape)
        
        total += labels.size(0) # label.size(0) is the batch size
        correct += (predicted == labels).sum().item() 
        # (predicted == labels).sum() would return how many of them are equal; 
        # .item() would make the tensor to the regular value
        
    print('Test Accuracy: %d %%' % (100 * correct / total))

  0%|          | 0/18 [00:00<?, ?it/s]

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

labels of validation: tensor([84, 82, 86, 87], device='cuda:1') torch.Size([4])
predicted of validation: tensor([76, 76, 76, 76], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 76,  70,  71, 119], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 76,  76,  76, 119], device='cuda:1') torch.Size([4])
labels of validation: tensor([87, 88, 90, 89], device='cuda:1') torch.Size([4])
predicted of validation: tensor([119,  71,  76,  76], device='cuda:1') torch.Size([4])
labels of validation: tensor([119,  78,  79,  77], device='cuda:1') torch.Size([4])
predicted of validation: tensor([119,  70,  76,  76], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 89, 100, 101, 105], device='cuda:1') torch.Size([4])
predicted of validation: tensor([76, 76, 76, 76], device='cuda:1') torch.Size([4])
total count: 28750
Epoch 0 | Train Loss: 3.0472 | Train Accuracy: 0.0783
labels of 0: tensor([ 71,  71, 119,  77], device='cuda:1') torch.Size([4])
predicted of 

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

labels of validation: tensor([84, 82, 86, 87], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 71,  76,  79, 103], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 76,  70,  71, 119], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 76,  76,  76, 119], device='cuda:1') torch.Size([4])
labels of validation: tensor([87, 88, 90, 89], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 76, 101,  76,  86], device='cuda:1') torch.Size([4])
labels of validation: tensor([119,  78,  79,  77], device='cuda:1') torch.Size([4])
predicted of validation: tensor([119, 104,  76,  76], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 89, 100, 101, 105], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 78, 100,  76, 101], device='cuda:1') torch.Size([4])
total count: 28750
Epoch 1 | Train Loss: 2.8304 | Train Accuracy: 0.0980
labels of 0: tensor([ 71,  71, 119,  77], device='cuda:1') torch.Size([4])
predi

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

labels of validation: tensor([84, 82, 86, 87], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 71,  76, 104, 104], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 76,  70,  71, 119], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 76,  76,  76, 119], device='cuda:1') torch.Size([4])
labels of validation: tensor([87, 88, 90, 89], device='cuda:1') torch.Size([4])
predicted of validation: tensor([79, 77, 79, 80], device='cuda:1') torch.Size([4])
labels of validation: tensor([119,  78,  79,  77], device='cuda:1') torch.Size([4])
predicted of validation: tensor([119, 103,  76,  76], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 89, 100, 101, 105], device='cuda:1') torch.Size([4])
predicted of validation: tensor([101,  89,  76, 100], device='cuda:1') torch.Size([4])
total count: 28750
Epoch 2 | Train Loss: 2.7741 | Train Accuracy: 0.1141
labels of 0: tensor([ 71,  71, 119,  77], device='cuda:1') torch.Size([4])
predicted

Training:   0%|          | 0/28750 [00:00<?, ?it/s]

labels of validation: tensor([84, 82, 86, 87], device='cuda:1') torch.Size([4])
predicted of validation: tensor([103,  76,  81,  85], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 76,  70,  71, 119], device='cuda:1') torch.Size([4])
predicted of validation: tensor([ 76, 101,  71, 119], device='cuda:1') torch.Size([4])
labels of validation: tensor([87, 88, 90, 89], device='cuda:1') torch.Size([4])
predicted of validation: tensor([70, 89, 90, 85], device='cuda:1') torch.Size([4])
labels of validation: tensor([119,  78,  79,  77], device='cuda:1') torch.Size([4])
predicted of validation: tensor([119,  88,  76,  76], device='cuda:1') torch.Size([4])
labels of validation: tensor([ 89, 100, 101, 105], device='cuda:1') torch.Size([4])
predicted of validation: tensor([81, 71, 82, 84], device='cuda:1') torch.Size([4])
total count: 28750
Epoch 3 | Train Loss: 2.7323 | Train Accuracy: 0.1231
labels of 0: tensor([ 71,  71, 119,  77], device='cuda:1') torch.Size([4])
predicted of 

In [ ]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))

model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)

criterion = nn.CrossEntropyLoss()
total_steps = 180

# save the best model
best_val_loss = float('inf')
patience = 500  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    count = 0 
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        
        count += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, count, type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()
        num_batches += 1
        
#     scheduler.step()
    add_log_msg(f"total count: {count}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for batched_g in dataloaders['valid']:
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, type=='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
#     print(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}')
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}')
    
    
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
#         torch.save(model.state_dict(), 'best_model.pth')
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f"../checkpoint_GAT/best_model_{epoch}.pt")
    
    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("Early stopping")
            break

            
# Testing Part
model.eval()
total = 0
correct = 0

with torch.no_grad():
    for data in dataloaders['test']:
        loss, accuracy, predicted = model_fn(data, model, criterion, device, type=='test')
        labels = data[1].to(device)  # Assuming labels are the second element in the tuple
        
#         print(f"labels: {labels}", labels.shape)
#         print(f"predicted: {predicted}", predicted.shape)
        
        add_log_msg(f"labels: {labels} {labels.shape}")
        add_log_msg(f"predicted: {predicted} {predicted.shape}")
        
        total += labels.size(0) # label.size(0) is the batch size
        correct += (predicted == labels).sum().item() 
        # (predicted == labels).sum() would return how many of them are equal; 
        # .item() would make the tensor to the regular value
        
#     print('Test Accuracy: %d %%' % (100 * correct / total))
add_log_msg(f'Test Accuracy: {100 * correct / total} %%')

Training:   9%|▊         | 5009/57500 [01:27<16:06, 54.33it/s]

08/16/2023, 10:31:31# labels of 5000: tensor([84, 82, 86, 87], device='cuda:0') torch.Size([4])
08/16/2023, 10:31:31# predicted of 5000: tensor([76, 76, 76, 77], device='cuda:0') torch.Size([4])


Training:  17%|█▋        | 10011/57500 [02:54<13:24, 59.04it/s]

08/16/2023, 10:32:58# labels of 10000: tensor([ 76,  70,  71, 119], device='cuda:0') torch.Size([4])
08/16/2023, 10:32:58# predicted of 10000: tensor([ 76,  76,  76, 119], device='cuda:0') torch.Size([4])


Training:  26%|██▌       | 15010/57500 [04:25<12:39, 55.96it/s]

...

Training:   9%|▊         | 5005/57500 [01:31<17:34, 49.80it/s]

08/16/2023, 10:48:34# labels of 5000: tensor([84, 82, 86, 87], device='cuda:0') torch.Size([4])
08/16/2023, 10:48:34# predicted of 5000: tensor([ 71,  76, 101, 104], device='cuda:0') torch.Size([4])


Training:  17%|█▋        | 10005/57500 [03:02<16:40, 47.45it/s]

08/16/2023, 10:50:06# labels of 10000: tensor([ 76,  70,  71, 119], device='cuda:0') torch.Size([4])
08/16/2023, 10:50:06# predicted of 10000: tensor([ 76,  76,  76, 119], device='cuda:0') torch.Size([4])


Training:  21%|██        | 12100/57500 [03:39<12:18, 61.50it/s]